In [3]:
import time
from datetime import datetime
import pygame
import os
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk

pygame.mixer.init() #Initialize pygame mixer for playing audio

class SpeakingClock:   
    
    def __init__(self,main_path,language='English',hour=0,minute=0,use_24_hour_format = True,use_reminder = True):
        self.main_path = main_path
        self.sub_folders={
            'English': 'English',
            'Portuguese': 'Portuguese',
            'Dongbei Mandarin': 'Dongbei Mandarin'
        }
        self.language = language
        self.hour = hour
        self.minute = minute
        self.use_24_hour_format = use_24_hour_format
        self.use_reminder = use_reminder
        
    def get_language_path(self):
        return os.path.join(self.main_path,self.sub_folders.get(self.language)) 
            
    def get_12_hour(self):
        return self.hour - 12 if self.hour > 12 else self.hour
        
    def next_hour(self):
        return 0 if self.hour in [11, 23] else (1 if self.hour in [0, 24] else (self.get_12_hour() + 1))
    
    def get_remaining_minutes(self):
        return 60 - self.minute       
    
    def announce_24_hour(self):
        output_path = self.get_language_path()
        files_to_play=[]
        files_to_play.append(os.path.join(output_path,"its.wav"))
        files_to_play.append(os.path.join(output_path,f"num{self.hour:02}.wav"))
        if self.minute == 0:
            files_to_play.append(os.path.join(output_path, "oclock.wav"))
        else:
            files_to_play.append(os.path.join(output_path, f"num{self.minute:02}.wav"))
        return files_to_play

    def announce_12_hour(self):
        output_path = self.get_language_path()
        in_12_hour = self.get_12_hour()
        next_hour=self.next_hour()
        remaining_minutes=self.get_remaining_minutes()
        files_to_play=[]
        files_to_play.append(os.path.join(output_path, "its.wav"))
        if self.minute == 0:
            files_to_play.append(os.path.join(output_path, f"num{in_12_hour:02}.wav"))
            files_to_play.append(os.path.join(output_path, "oclock.wav"))
        elif self.minute == 30:
            files_to_play.append(os.path.join(output_path, "halfpast.wav"))
            files_to_play.append(os.path.join(output_path, f"num{in_12_hour:02}.wav"))
        elif self.minute == 15:
            files_to_play.append(os.path.join(output_path, "quarter.wav"))
            files_to_play.append(os.path.join(output_path, "past.wav"))
            files_to_play.append(os.path.join(output_path, f"num{in_12_hour:02}.wav"))
        elif self.minute == 45:
            files_to_play.append(os.path.join(output_path, "quarter.wav"))
            files_to_play.append(os.path.join(output_path, "to.wav"))
            files_to_play.append(os.path.join(output_path, f"num{next_hour:02}.wav"))
        elif self.minute < 30:
            files_to_play.append(os.path.join(output_path, f"num{self.minute:02}.wav"))
            files_to_play.append(os.path.join(output_path, "past.wav"))
            files_to_play.append(os.path.join(output_path, f"num{in_12_hour:02}.wav"))
        else:
            files_to_play.append(os.path.join(output_path, f"num{remaining_minutes:02}.wav"))
            files_to_play.append(os.path.join(output_path, "to.wav"))
            files_to_play.append(os.path.join(output_path, f"num{next_hour:02}.wav"))
        if 0 <= self.hour < 12:
            files_to_play.append(os.path.join(output_path, "am.wav"))
        else:
            files_to_play.append(os.path.join(output_path, "pm.wav"))
        return files_to_play
        
    def get_reminder(self):
        output_path = self.get_language_path()
        reminders = {
            (0,6): "additional01.wav",
            (6,10): "additional02.wav",
            (11,13): "additional03.wav",
            (17,19): "additional04.wav",
            (19,21): "additional05.wav",
            (21,24): "additional06.wav"
        }
        for (start_hour, end_hour), reminder_audio in reminders.items():
            if start_hour <= self.hour < end_hour:
                return[os.path.join(output_path, reminder_audio)]
        return []
        
    def play_audio(self, file_paths):
        for file_path in file_paths:
            pygame.mixer.music.load(file_path)
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                time.sleep(0.1)  # Shorter sleep to avoid blocking 
                   
    def speak_time(self):
        files_to_play=[]
        if self.use_24_hour_format:
            files_to_play.extend(self.announce_24_hour())
        else:
            files_to_play.extend(self.announce_12_hour())
        if self.use_reminder:
            reminder_files = self.get_reminder()
            if reminder_files:
                files_to_play.extend(reminder_files)
        self.play_audio(files_to_play) 
        files_to_play=[]

   
class English(SpeakingClock):
    
    def __init__(self,main_path,hour=0,minute=0,use_24_hour_format=True, use_reminder=True):
        super().__init__(main_path, language='English', hour=hour, minute=minute, use_24_hour_format=use_24_hour_format, use_reminder=use_reminder)


class Portuguese(SpeakingClock):
    
    def __init__(self,main_path,hour=0,minute=0,use_24_hour_format=True, use_reminder=True):
        super().__init__(main_path, language='Portuguese', hour=hour, minute=minute, use_24_hour_format=use_24_hour_format, use_reminder=use_reminder)
      
    def announce_24_hour(self):
        output_path = self.get_language_path()  
        files_to_play = []
        if self.hour == 0 and self.minute == 0:
            files_to_play.append(os.path.join(output_path, "12midnight.wav"))
        elif self.hour == 12 and self.minute == 0:
            files_to_play.append(os.path.join(output_path, "12noon.wav"))
        else:
            files_to_play.append(os.path.join(output_path, "its.wav"))
            files_to_play.append(os.path.join(output_path, f"num{self.hour:02}.wav"))
            if self.minute != 0:
                files_to_play.append(os.path.join(output_path, "and.wav"))
                files_to_play.append(os.path.join(output_path, f"num{self.minute:02}.wav")) 
        return files_to_play
        
    def announce_12_hour(self):
        output_path = self.get_language_path()
        in_12_hour = self.get_12_hour()
        files_to_play = []
        if in_12_hour == 0 and self.minute == 0:
            files_to_play.append(os.path.join(output_path, "12midnight.wav"))
        elif in_12_hour == 12 and self.minute == 0:
            files_to_play.append(os.path.join(output_path, "12noon.wav"))
        elif in_12_hour == 1 and self.minute == 0:
            files_to_play.append(os.path.join(output_path, "Its_1_oclock.wav"))
        else:
            files_to_play.append(os.path.join(output_path, "its.wav"))
            files_to_play.append(os.path.join(output_path, f"num{in_12_hour:02}.wav")) 
            if self.minute != 0:
                files_to_play.append(os.path.join(output_path, "and.wav"))
                files_to_play.append(os.path.join(output_path, f"num{self.minute:02}.wav"))    
        if not (self.hour == 0 and self.minute == 0) and not (self.hour == 12 and self.minute == 0):
            if 0 <= self.hour < 12:
                files_to_play.append(os.path.join(output_path, "am.wav"))
            elif 12 < self.hour <= 18:
                files_to_play.append(os.path.join(output_path, "pm1.wav")) 
            else:
                files_to_play.append(os.path.join(output_path, "pm2.wav"))
        return files_to_play

class DongbeiMandarin(SpeakingClock):

    def __init__(self,main_path,hour=0,minute=0,use_24_hour_format=True, use_reminder=True):
        super().__init__(main_path, language='Dongbei Mandarin', hour=hour, minute=minute, use_24_hour_format=use_24_hour_format, use_reminder=use_reminder)
    
    def announce_24_hour(self):  
        output_path = self.get_language_path()  
        files_to_play = []
        files_to_play.append(os.path.join(output_path, "its.wav"))
        files_to_play.append(os.path.join(output_path, f"num{self.hour:02}.wav"))  
        files_to_play.append(os.path.join(output_path, "oclock.wav"))
        if self.minute > 0:
            if self.minute < 10:
                files_to_play.append(os.path.join(output_path, "num00.wav"))  # For single-digit minute, play "zero"
            files_to_play.append(os.path.join(output_path, f"num{self.minute:02}.wav"))
            files_to_play.append(os.path.join(output_path, "minute.wav"))  # Play the "minute" sound
        return files_to_play
        
    def announce_12_hour(self):
        output_path = self.get_language_path()  
        in_12_hour = self.get_12_hour()
        files_to_play = []
        files_to_play.append(os.path.join(output_path, "its.wav"))
        if 0 <= self.hour < 12:
            files_to_play.append(os.path.join(output_path, "morning.wav"))
        elif self.hour == 12:
            files_to_play.append(os.path.join(output_path, "noon.wav")) 
        elif 12 < self.hour <= 18:
            files_to_play.append(os.path.join(output_path, "afternoon.wav")) 
        else:
            files_to_play.append(os.path.join(output_path, "evening.wav"))
        files_to_play.append(os.path.join(output_path, f"num{in_12_hour:02}.wav"))
        files_to_play.append(os.path.join(output_path, "oclock.wav"))
        if self.minute == 15:
            files_to_play.append(os.path.join(output_path, "num01.wav"))
            files_to_play.append(os.path.join(output_path, "quarter.wav"))
        elif self.minute == 45:
            files_to_play.append(os.path.join(output_path, "num03.wav"))
            files_to_play.append(os.path.join(output_path, "quarter.wav"))
        elif self.minute == 30:
            files_to_play.append(os.path.join(output_path, "halfpast.wav"))
        elif self.minute > 0:
            if self.minute < 10:
                files_to_play.append(os.path.join(output_path, "num00.wav"))  # For single-digit minute, play "zero"
            files_to_play.append(os.path.join(output_path, f"num{self.minute:02}.wav"))
            files_to_play.append(os.path.join(output_path, "minute.wav"))  # Play the "minute" sound
        return files_to_play



class TalkingClockUI:
    def __init__(self, root, main_path):
        self.root = root
        self.main_path = main_path
        self.root.title("Talking Clock")
        self.root.geometry("819x574")
        
       # Set background image
        self.background = Image.open(os.path.join(main_path, "background.png"))
        self.background_photo = ImageTk.PhotoImage(self.background)
        self.background_label = tk.Label(root, image=self.background_photo)
        self.background_label.pack(fill=tk.BOTH, expand=True)
        self.root.bind("<Configure>", self.resize_background) 

        # Language Dropdown
        self.language_label = tk.Label(root, text="Language:", bg="#87CEFA")
        self.language_label.place(x=900, y=60)
        self.language_var = tk.StringVar()
        self.language_dropdown = ttk.Combobox(root, textvariable=self.language_var)
        self.language_dropdown['values'] = ("English", "Portuguese", "Dongbei Mandarin")
        self.language_dropdown.current(0)
        self.language_dropdown.place(x=1000, y=60)

        # 12-hour or 24-hour format Radio Buttons
        self.hour_format_label = tk.Label(root, text="12/24-hour:", bg="#87CEFA")
        self.hour_format_label.place(x=900, y=100)
        self.hour_format_var = tk.StringVar(value="24_hour")
        self.hour_24_radio = tk.Radiobutton(root, text="24-hour", variable=self.hour_format_var, value="24_hour", bg="#87CEFA")
        self.hour_24_radio.place(x=1000, y=100)
        self.hour_12_radio = tk.Radiobutton(root, text="12-hour", variable=self.hour_format_var, value="12_hour", bg="#87CEFA")
        self.hour_12_radio.place(x=1100, y=100)

        # Real time or manual time Radio Buttons
        self.time_type_label = tk.Label(root, text="Real/Manual:", bg="#87CEFA")
        self.time_type_label.place(x=900, y=180)
        self.time_type_var = tk.StringVar(value="real_time")
        self.real_time_radio = tk.Radiobutton(root, text="Real time", variable=self.time_type_var, value="real_time", bg="#87CEFA")
        self.real_time_radio.place(x=1000, y=180)
        self.manual_time_radio = tk.Radiobutton(root, text="Manual time", variable=self.time_type_var, value="manual_time", bg="#87CEFA")
        self.manual_time_radio.place(x=1100, y=180)

        # Manual Time Input (Initially Hidden)
        self.hour_label = tk.Label(root, text="Hour (0-23):", bg="#87CEFA")
        self.minute_label = tk.Label(root, text="Minute (0-59):", bg="#87CEFA")
        self.hour_var = tk.IntVar()
        self.minute_var = tk.IntVar()
        self.hour_entry = tk.Entry(root, textvariable=self.hour_var)
        self.minute_entry = tk.Entry(root, textvariable=self.minute_var)
        
        # Set initial state of entry boxes
        self.toggle_manual_time()

        # Set initial state of entry boxes
        self.time_type_var.trace("w", lambda *args: self.toggle_manual_time())
        
        # Validate input
        self.hour_entry.bind("<KeyRelease>", self.validate_hour)
        self.minute_entry.bind("<KeyRelease>", self.validate_minute)
        
        # Reminder Radio Buttons
        self.reminder_label = tk.Label(root, text="Reminder:", bg="#87CEFA")
        self.reminder_label.place(x=900, y=140)
        self.reminder_var = tk.StringVar(value="yes")
        self.reminder_yes_radio = tk.Radiobutton(root, text="Yes", variable=self.reminder_var, value="yes", bg="#87CEFA")
        self.reminder_yes_radio.place(x=1000, y=140)
        self.reminder_no_radio = tk.Radiobutton(root, text="No", variable=self.reminder_var, value="no", bg="#87CEFA")
        self.reminder_no_radio.place(x=1100, y=140)

        # Real-time Clock Display
        self.time_display_label = tk.Label(root, text="", font=("Comic Sans MS", 30, "bold"), fg="black", bg="#FFFFE0")
        self.time_display_label.place(x=250, y=290)
        self.update_time_display()

        # Reset Button
        self.reset_button = tk.Button(root, text="Reset", command=self.reset_clock, bg="#FFA500")
        self.reset_button.place(x=900, y=300)
    
        # Announce Button to simulate action
        self.announce_button = tk.Button(root, text="Announce Time", command=self.announce_time, bg="#32CD32")
        self.announce_button.place(x=900, y=340)

    def toggle_manual_time(self):
        if self.time_type_var.get() == "manual_time":
            self.hour_label.place(x=900, y=220)
            self.hour_entry.place(x=1000, y=220)
            self.minute_label.place(x=900, y=260)
            self.minute_entry.place(x=1000, y=260)
        else:
            self.hour_label.place_forget()
            self.hour_entry.place_forget()
            self.minute_label.place_forget()
            self.minute_entry.place_forget()

    def validate_hour(self, event):
        if self.hour_var.get() < 0 or self.hour_var.get() > 23:
            self.hour_var.set(0)  # Reset to 0 if out of range

    def validate_minute(self, event):
        if self.minute_var.get() < 0 or self.minute_var.get() > 59:
            self.minute_var.set(0)  # Reset to 0 if out of range
            
    def announce_time(self):
        language = self.language_var.get()
        use_24_hour_format = self.hour_format_var.get() == "24_hour"
        use_reminder = self.reminder_var.get() == "yes"
        
        if self.time_type_var.get() == "real_time":
            now = datetime.now()
            hour=now.hour
            minute=now.minute
        else:
            try:
                hour = int(self.hour_entry.get())
                minute = int(self.minute_entry.get())
            except ValueError:
                print("Please enter valid hour and minute values.")
                return
                
        if language == "English":
            clock = English(self.main_path, hour=hour, minute=minute, use_24_hour_format=use_24_hour_format, use_reminder=use_reminder)
        elif language == "Portuguese":
            clock = Portuguese(self.main_path, hour=hour, minute=minute, use_24_hour_format=use_24_hour_format, use_reminder=use_reminder)
        elif language == "Dongbei Mandarin":
            clock = DongbeiMandarin(self.main_path, hour=hour, minute=minute, use_24_hour_format=use_24_hour_format, use_reminder=use_reminder)
        
        clock.speak_time() 
        
    def update_time_display(self):
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        self.time_display_label.config(text=f"{current_time}")
        self.root.after(1000, self.update_time_display)

    def reset_clock(self):
        self.hour_entry.delete(0, tk.END)
        self.minute_entry.delete(0, tk.END)
        self.language_var.set("English")
        self.hour_format_var.set("24_hour")
        self.time_type_var.set("real_time")
        self.reminder_var.set("yes")
        self.update_time_display()

    def resize_background(self, event):
        # Resize the image to fit the window
        new_width = event.width
        new_height = event.height
        aspect_ratio = self.original_background.width / self.original_background.height
        if new_width / new_height > aspect_ratio:
            # Width is greater than height; limit by height
            new_width = int(new_height * aspect_ratio)
        else:
            # Height is greater than width; limit by width
            new_height = int(new_width / aspect_ratio)
        resized_image = self.original_background.resize((new_width, new_height), Image.LANCZOS)
        self.background_photo = ImageTk.PhotoImage(resized_image)
        self.background_label.config(image=self.background_photo)
    
    
if __name__ == "__main__":
    main_path = "/Users/Documents/1a_SpeakingClock/Final"
    root = tk.Tk()
    app = TalkingClockUI(root, main_path)
    root.mainloop()
    

        

Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/tkinter/__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/7c/1hkply_n6b1c97m9785lcp_c0000gn/T/ipykernel_81122/896165203.py", line 367, in resize_background
    aspect_ratio = self.original_background.width / self.original_background.height
                   ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'TalkingClockUI' object has no attribute 'original_background'. Did you mean: 'resize_background'?
invalid command name "4768353664update_time_display"
    while executing
"4768353664update_time_display"
    ("after" script)
Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/tkinter/__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/7c/1hkply_n6b1c97m9785lcp_c0000gn/T/ipykernel_81122/896165203.py", line 367, in res

In [59]:
!pip install pillow

In [6]:
pip install pyimage

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/mmseg-1.3.0-py3.12-macosx-11.1-arm64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.
